# Challenge 1 Data: Smart meters in London

## Objectives 

### Overall

The objective of this project is to curate information about monitoring electricity consumption patterns using statistical and machine learning methods. 

Facilitate collaboration between statisticians, computer scientists, data scientists and the wider community.

### Challenge specific objectives 1

https://github.com/cobleg/Hack-A-Gig/wiki

Derive insight from smart meter data. 

Use this insight to generate a seven day ahead forecast for each consumer. 

Provide advice to consumers on why their demand for electricity varies over the forecast week.



### Challenge specific objectives 2

https://github.com/cobleg/Hack-A-Gig/wiki/Challenge-1-Data:-Smart-meters-in-London

Develop an understanding of the consumption patterns contained in the data.

Relate the consumption patterns to statistically distinct groups of consumers.

Create a scalable forecasting process to accurately predict consumption for each group and individual household for up to a week ahead.

### Research questions

Are there any obvious seasonal (or cyclical) patterns. If so, are the cycles daily, weekly, annual etc.

The ratio of peak to average demand - known as load factor in the power industry.

Any trends evident - growth or contraction.

The degree of correlation across consumers.

How large is the ratio of the signal to noise? That is, are the identified consumption patterns highly predictable?

## Prepare data for exploratory analysis

Link data files to facilitate exploration of relationships between variables.

Identify and flag missing values, outliers.

Create time index, binary variables for the time of day, week and year.

create a tidy data set (for infomration see: https://vita.had.co.nz/papers/tidy-data.pdf)

## Explore the data

Create a table of summary statistics (i.e. min, mean, median, max, std. dev., range, the coefficient of variation) for both the time-series and structural variables.

Create line plots for time-series data over the following intervals: 24 hours of the day; days of the week; months of the year.

Create scatter plots of temperature, wind etc. versus electricity consumption.

Create other visualisations of the data to understand the impact of consumer segmentation on electricity consumption.

Create clustering visualisation to show interesting patterns in the data.

Apply any unsupervised learning methods to better understand the patterns in the data.

In [1]:
from fastai.structured import *
from fastai.column_data import *

In [2]:
PATH='../input/merged_data/'

Dataset background information can be found here:
    
http://jmdaignan.com/2018/01/28/Londonsmartmeter/

Kaggle link:

https://www.kaggle.com/jeanmidev/smart-meters-in-london/home
        

### HHBlock data

* The hhblock_dataset that contains the transpose data of a day for one household (as an array) with for example the hh_0 column is the consumption between 00:00 and 00:30

In [3]:
#merge datasets
#df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('../input/hhblock_dataset/', "block_*.csv"))))
#df.to_csv('../input/merged_data/hhblock_all.csv')

### Daily data

* The daily_dataset that contains daily informations on the consumption of the households



In [4]:
#concatenate all daly data
#daily_all = pd.concat(map(pd.read_csv, glob.glob(os.path.join('../input/daily_dataset/', "block_*.csv"))))
#daily_all.to_csv('../input/merged_data/daily_all.csv')

### Half hourly data

* LCLid that corresponds to the household id

* tstp the timestamp of the measure

* energy(kWh/hh) the energy consumes in the past 30 minutes in kWh

### informations_households

* LCLid that correspond to the household id

* stdorToU the kind of tariff applied (ToU the dynamic tariff in function of the days or Std the classic fixed tariff)

* Acorn the ACORN group associated, that categorises the household

* Acorn_grouped this is another more global classification of the ACORN (fusion of different ACORN groups)

* file name of the file in the different zip files where you can find the data of the household

### acorn_details

* contains the index for multiple parameters in comparison of the national (that have an index of 100)

https://acorn.caci.co.uk/downloads/Acorn-User-guide.pdf

Acorn is a segmentation tool which categorises the UK’s population into demographic types. Acorn segments households, postcodes and neighbourhoods into 6 categories, 18 groups and 62 types.

In [5]:
#concatenate all 1/2 hourly data
#halfhourly_all = pd.concat(map(pd.read_csv, glob.glob(os.path.join('../input/halfhourly_dataset/', "block_*.csv"))))
#halfhourly_all.to_csv('../input/merged_data/halfhourly_all.csv')

In [6]:
table_names = ['daily_all', 'halfhourly_all', 'hhblock_all', 'acorn_details', 
               'informations_households', 'uk_bank_holidays', 
               'weather_daily_darksky', 'weather_hourly_darksky']

In [7]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]

In [8]:
from IPython.display import HTML, display

In [9]:
for t, name in zip(tables, table_names): 
    print(name)
    display(t.head(n=2))

daily_all


,Unnamed: 0,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,0,MAC000041,2011-12-08,0.295,0.396259,1.071,27,0.285051,10.699,0.119
1,1,MAC000041,2011-12-09,0.204,0.235437,0.744,48,0.184686,11.301,0.023


halfhourly_all


,Unnamed: 0,LCLid,tstp,energy(kWh/hh)
0,0,MAC000041,2011-12-08 10:30:00.0000000,0.126
1,1,MAC000041,2011-12-08 11:00:00.0000000,0.12


hhblock_all


...

,Bank holidays,Type
0,2012-12-26,Boxing Day
1,2012-12-25,Christmas Day


weather_daily_darksky


,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,...,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase
0,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,...,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52
1,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,...,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53


weather_hourly_darksky


,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy


In [10]:
for t, name in zip(tables, table_names): 
    print(name)
    display(DataFrameSummary(t).summary())

daily_all


,Unnamed: 0,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
count,3.51043e+06,NaN,NaN,3.5104e+06,3.5104e+06,3.5104e+06,3.51043e+06,3.4991e+06,3.5104e+06,3.5104e+06
mean,15791.6,NaN,NaN,0.158739,0.21173,0.834521,47.8036,0.172667,10.1241,0.0596258
std,9203.25,NaN,NaN,0.170186,0.190846,0.668316,2.81098,0.153121,9.12879,0.0870131
min,0,NaN,NaN,0,0,0,0,0,0,0
25%,7835,NaN,NaN,0.067,0.0980833,0.346,48,0.0691163,4.682,0.02
50%,15723,NaN,NaN,0.1145,0.163292,0.688,48,0.132791,7.815,0.039
75%,23629,NaN,NaN,0.191,0.262458,1.128,48,0.229312,12.569,0.071
max,36167,NaN,NaN,6.9705,6.92825,10.761,48,4.02457,332.556,6.524
counts,3510433,3510433,3510433,3510403,3510403,3510403,3510433,3499102,3510403,3510403
uniques,36168,5566,829,10437,421337,6425,44,3275190,401153,2149


halfhourly_all


,Unnamed: 0,LCLid,tstp,energy(kWh/hh)
count,1.67817e+08,NaN,NaN,NaN
mean,754967,NaN,NaN,NaN
std,439993,NaN,NaN,NaN
min,0,NaN,NaN,NaN
25%,374591,NaN,NaN,NaN
50%,751675,NaN,NaN,NaN
75%,1.12964e+06,NaN,NaN,NaN
max,1.73057e+06,NaN,NaN,NaN
counts,167817021,167817021,167817021,167817021
uniques,1730575,5566,40405,9611


hhblock_all


...

,Bank holidays,Type
count,25,25
unique,25,11
top,2014-12-25,Christmas Day
freq,1,3
counts,25,25
uniques,25,11
missing,0,0
missing_perc,0%,0%
types,unique,categorical


weather_daily_darksky


,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,...,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase
count,882,NaN,882,NaN,882,NaN,881,882,882,NaN,...,882,NaN,NaN,NaN,NaN,NaN,882,NaN,NaN,882
mean,13.6601,NaN,195.703,NaN,6.53003,NaN,0.477605,3.5818,1014.13,NaN,...,13.5424,NaN,NaN,NaN,NaN,NaN,5.73804,NaN,NaN,0.50093
std,6.18274,NaN,89.3408,NaN,4.83088,NaN,0.193514,1.69401,11.073,NaN,...,6.2602,NaN,NaN,NaN,NaN,NaN,6.04875,NaN,NaN,0.287022
min,-0.06,NaN,0,NaN,-7.84,NaN,0,0.2,979.25,NaN,...,-0.81,NaN,NaN,NaN,NaN,NaN,-8.88,NaN,NaN,0
25%,9.5025,NaN,120.5,NaN,3.18,NaN,0.35,2.37,1007.43,NaN,...,9.2125,NaN,NaN,NaN,NaN,NaN,1.105,NaN,NaN,0.26
50%,12.625,NaN,219,NaN,6.38,NaN,0.47,3.44,1014.62,NaN,...,12.47,NaN,NaN,NaN,NaN,NaN,4.885,NaN,NaN,0.5
75%,17.92,NaN,255,NaN,10.0575,NaN,0.6,4.5775,1021.75,NaN,...,17.91,NaN,NaN,NaN,NaN,NaN,11.2775,NaN,NaN,0.75
max,32.4,NaN,359,NaN,17.77,NaN,1,9.96,1040.92,NaN,...,32.4,NaN,NaN,NaN,NaN,NaN,20.54,NaN,NaN,0.99
counts,882,882,882,882,882,882,881,882,882,882,...,882,882,882,881,882,882,882,882,882,882
uniques,711,882,304,6,687,882,96,466,802,882,...,715,882,882,881,88,882,706,882,882,100


weather_hourly_darksky


,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
count,21165,21165,21165,NaN,21165,21152,21165,21165,NaN,NaN,21165,NaN
mean,11.1665,195.686,10.4715,NaN,6.5305,1014.13,9.23034,3.90522,NaN,NaN,0.781829,NaN
std,3.09934,90.6295,5.7819,NaN,5.04197,11.3883,6.94092,2.02685,NaN,NaN,0.140369,NaN
min,0.18,0,-5.64,NaN,-9.98,975.74,-8.88,0.04,NaN,NaN,0.23,NaN
25%,10.12,121,6.47,NaN,2.82,1007.43,3.9,2.42,NaN,NaN,0.7,NaN
50%,12.26,217,9.93,NaN,6.57,1014.78,9.36,3.68,NaN,NaN,0.81,NaN
75%,13.08,256,14.31,NaN,10.33,1022.05,14.32,5.07,NaN,NaN,0.89,NaN
max,16.09,359,32.4,NaN,19.88,1043.32,32.42,14.8,NaN,NaN,1,NaN
counts,21165,21165,21165,21165,21165,21152,21165,21165,21165,21165,21165,21165
uniques,953,360,2803,21165,2398,4988,3124,1095,2,7,78,13


### Join data

join_df is a function for joining tables on specific fields. By default, we'll be doing a left outer join of right on the left argument using the given fields for each table.

Pandas does joins using the merge method. The suffixes argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "_y" to those on the right.

**This is pretty memory hungry**

In [11]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [15]:
halfhourly_combined = join_df(tables[0], tables[2], "LCLid", "LCLid") 

MemoryError: 

In [ ]:
halfhourly_combined.head(n=2)

In [ ]:
halfhourly_combined = join_df(halfhourly_combined, table[4], "LCLid", "LCLid")

In [ ]:
halfhourly_combined.to_csv('../input/merged_data/halfhourly_combined.csv')